# Autoria

* Cristiano Sampaio Pinheiro, RA 256352
* Jhonatan Cléto, RA 256444
* Mylena Roberta dos Santos, RA 222687

# Bibliotecas e Paths

In [ ]:
!pip install glrlm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd

from PIL import Image
from scipy import stats

from glrlm import GLRLM
from glrlm.normalizer import Normalizer
from glrlm.degree import Degree
from glrlm.operator import Operator

from skimage.feature import greycomatrix, greycoprops
from skimage.feature import local_binary_pattern

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

PATH = '/content/drive/MyDrive/MC936/P4/'
PATH_DATA = PATH + 'data/'
PATH_TRAIN = PATH_DATA + 'Train/'
PATH_LES = PATH_DATA + 'LES/'
PATH_ASSETS = PATH + 'assets/'

Mounted at /content/drive


# Obtem Imagens Previamente Selecionadas

Partindo do dataframe gerado após a seleção das imagens é construída uma lista de imagens com suas respectivas máscaras, tal estrutura é salva em dicionários para cada lesão com uma chave para cada paciente.

In [ ]:
def img_dict(imgs:list, masks:list, areas:list) -> dict:
  """
    ## Descrição:
     - Constrói um dicionário com as imagens, máscaras e
     tamanho das máscaras de cada paciente no conjunto

    ## Entrada:
     - imgs: Lista com os paths de todas as imagens
     do conjunto
     - masks: Lista com os paths das máscaras do
     conjunto
     - areas: Lista com o tamanho das máscaras do
     conjunto
    
    ## Saída:
     - img_dict: dicionário em que as chaves são os ids
     dos pacientes e os valores são dicionários contendo
     listas das imagens, máscaras e áreas das máscaras
     dos cortes selecionadas para cada paciente
  """

  img_dict = dict()

  for i in range(len(imgs)):
    imp = imgs[i]
    pnum = imp[:3] # obtem o id do paciente

    if pnum in img_dict.keys():
      img_dict[pnum]['imgs'].append(imp)  
      img_dict[pnum]['masks'].append(masks[i])
      img_dict[pnum]['areas'].append(areas[i])

    else:
      dt = {
            'imgs': [imp],
            'masks': [masks[i]],
            'areas': [areas[i]]
           }
      
      img_dict[pnum] = dt

  return img_dict

In [ ]:
# Abrir csvs pré-computados
avc_set_df = pd.read_csv(PATH_DATA + "avc_set_df.csv")
em_set_df = pd.read_csv(PATH_DATA + "em_set_df.csv")
les_set_df = pd.read_csv(PATH_DATA + "les_set_df.csv")

# Retira máscaras totalmente pretas
values = ['610_FLAIR26_mask.png', '611_FLAIR15_mask.png', '611_FLAIR16_mask.png']
les_set_df = les_set_df[les_set_df.masks.isin(values) == False]
les_set_df.reset_index(inplace=True)

# Obter lista de imagens
avc_imgs = avc_set_df['imgs']
avc_masks = avc_set_df['masks']
avc_areas = avc_set_df['areas']
em_imgs = em_set_df['imgs']
em_masks = em_set_df['masks']
em_areas = em_set_df['areas']
les_imgs = les_set_df['imgs']
les_masks = les_set_df['masks']
les_areas = les_set_df['areas']

# Dicionários com as imagens separadas por paciente
avc_dict = img_dict(avc_imgs, avc_masks, avc_areas)
em_dict = img_dict(em_imgs, em_masks, em_areas)
les_dict = img_dict(les_imgs, les_masks, les_areas)

print("Num Pacientes AVC: ", len(avc_dict))
print("Num Pacientes EM: ", len(em_dict))
print("Num Pacientes LES: ", len(les_dict))

Num Pacientes AVC:  50
Num Pacientes EM:  51
Num Pacientes LES:  78


# Aplica Máscaras

As máscara são aplicadas nas imagens através de uma simples mutiplicação de matrizes, para eliminar a grande quantidade de preto resultante dessa operação realizamos um crop na imagem. Os limites para esse corte são determinados pelos primeiros pixels com cor diferente de preto.

In [ ]:
# Função para aplicação das máscaras e crop das imagens de maneira individual
from typing import Tuple

def get_crop_limits(mask_mat: np.array) -> Tuple[int, int, int, int]:
    """
      ## Descrição:
      - Obtem as coordenadas dos limites das lesões através da máscara

      ## Entrada:
      - mask_mat: np.array com a máscara

      ## Saída:
      - min_y: inteiro correspondente a coordenada do limite inferior 
      - min_x: inteiro correspondente a coordenada do limite esquerdo 
      - max_y: inteiro correspondente a coordenada do limite superior 
      - max_x: inteiro correspondente a coordenada do limite direito 
    """
    # Obtem índices de elementos diferentes de 0
    white_pt_coords=np.argwhere(mask_mat)

    # Obtem limites
    min_y = min(white_pt_coords[:,0])
    min_x = min(white_pt_coords[:,1])
    max_y = max(white_pt_coords[:,0])
    max_x = max(white_pt_coords[:,1])
    
    return min_y, min_x, max_y, max_x


def apply_mask(img_name: str, mask_name: str, path: str, group='') -> np.array:
  """
    ## Descrição:
     - Abre uma imagem e sua respectiva máscara, as converte para np.array e
     realiza a multiplicação dos arrays já aplicando um crop no resultado

    ## Entrada:
     - img_name: String contendo o nome da imagem
     - mask_name: String contendo o nome da máscara
     - path: String contendo o caminho para importar o arquivo
     - group: String usada para compor o caminho para acessar imagens de 
     diferentes grupos

    ## Saída:
     - crop_mask_img: np.array contendo o resultado da multiplicação dos arrays 
     devidamente dimensionado
  """

  # Abre imagens
  img = Image.open(path + group + '/' + img_name)
  mask = Image.open(path + group + '/' + mask_name)

  # Converte imagens para array
  img_mat = np.array(img, dtype=np.uint8)
  mask_mat = np.array(mask, dtype=np.uint8)

  if(path!=PATH_LES):
    mask_mat = mask_mat-253 # Realiza correção em algumas máscaras

  # Obtem limites para o crop da imagem
  min_y, min_x, max_y, max_x = get_crop_limits(mask_mat)

  # Realiza mutiplicação e delimita imagem resultante
  crop_mask_img = (mask_mat * img_mat)[min_y:max_y,min_x:max_x]

  return crop_mask_img

In [ ]:
# Cria dicionários com os ROIs das imagens
avc_roi_dict = {}
em_roi_dict = {}
les_roi_dict = {}

# Percorre dicionários com as informações referentes aos pares imagem/máscara gerando os ROIs 
for patient in avc_dict.keys():
  aux_imgs = []
  aux_areas = []
  for i in range(len(avc_dict[patient]['imgs'])):
    aux_imgs.append(apply_mask(avc_dict[patient]['imgs'][i], avc_dict[patient]['masks'][i], PATH_TRAIN, 'AVC'))
    aux_areas.append(avc_dict[patient]['areas'][i])
  avc_roi_dict[patient] = {'roi_imgs':aux_imgs, 'areas': aux_areas}

for patient in em_dict.keys():
  aux_imgs = []
  aux_areas = []
  for i in range(len(em_dict[patient]['imgs'])):
    aux_imgs.append(apply_mask(em_dict[patient]['imgs'][i], em_dict[patient]['masks'][i], PATH_TRAIN, 'EM'))
    aux_areas.append(em_dict[patient]['areas'][i])
  em_roi_dict[patient] = {'roi_imgs':aux_imgs, 'areas': aux_areas}

for patient in les_dict.keys():
  aux_imgs = []
  aux_areas = []
  for i in range(len(les_dict[patient]['imgs'])):
    aux_imgs.append(apply_mask(les_dict[patient]['imgs'][i], les_dict[patient]['masks'][i], PATH_LES))
    aux_areas.append(les_dict[patient]['areas'][i])
  les_roi_dict[patient] = {'roi_imgs':aux_imgs, 'areas': aux_areas}

# Extração de Atributos

Ainda que a quantidade de pixels pretos nas imagens tenha diminuído após a aplicação das máscaras, executamos os métodos de extração de features de maneira a não considerar esses pixels. Como as lesões estão concentradas próximas à cor branca, a princípio os pixels pretos não agregarão na identificação e classificação dessas lesões, ainda, no caso de múltiplas lesões com um grande espaçamento, o background pode trazer um ruído considerável para os dados obtidos.

## Extraindo Atributos de Histogramas

Os histogramas das imagens foram confeccionados utilizando a função _[histogram](https://numpy.org/doc/stable/reference/generated/numpy.histogram.html)_ da biblioteca _numpy_, a qual computa um histograma a partir de um dataset, no caso um numpy.array. Os pixels pretos foram eliminados do próprio array, de modo que o valor 0 foi desconsiderado na montagem do histograma. 

Com os histogramas montados suas features foram extraídas utilizando a função _[stats.describe](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.describe.html)_ da biblioteca _scipy_. Com isso foi possível obter as informações de nobs, minmax, mean, variance, skewness e kurtosis.


In [ ]:
# Funções para montar e extrair features relacionadas aos histogramas das imagens
def make_hist(imgs: list) -> list:
  """
    ## Descrição:
     - Obtem lista de histogramas a partir das imagens recebidas

    ## Entrada:
     - imgs: Lista de np.array correspondentes aos ROIs das imagens
    
    ## Saída:
     - htgs: Lista de dicionários contendo o histograma e bins
     para cada imagem
  """

  # Cria estrutura para armazenar histogramas
  htgs = []
  nbins = 20

  # Percorre imagens gerando histogramas
  for i in range(len(imgs)):
    # Elimina pixels pretos da imagem
    img = imgs[i][imgs[i]>0]
    # Confecciona histograma
    h, bin_edges = np.histogram(img, nbins,(0,255))
    htgs.append({'hist': h, 'bins': bin_edges})

  return htgs

def get_hists_features(imgs: list) -> list:
  """
    ## Descrição:
     - Obtem lista de features provenientes de histogramas 
     construídos a partir das imagens recebidas

    ## Entrada:
     - imgs: Lista de np.array correspondentes aos ROIs das imagens
    
    ## Saída:
     - hists_features: Lista de dicionários contendo as features dos histogramas
  """

  # Cria estrutura para armazenar as features dos histogramas
  hists_features = []
  hists = make_hist(imgs)
  # Percorre histrogramas de um conjunto de imagens extraindo suas features
  for i in range(len(hists)):
    describe = stats.describe(hists[i]['hist'])._asdict()
    hists_features.append( {'min': describe['minmax'][0],
                            'max': describe['minmax'][1],
                            'mean': describe['mean'],
                            'variance': describe['variance'],
                            'skewness': describe['skewness'],
                            'kurtosis': describe['kurtosis']} )
  
  return hists_features

## Extraindo Atributos de Textura

Dos atributos de textura, fizemos uso da Matriz de Coocorrência (GLCM), Matriz de Comprimento de Corrida (GLRLM) e Local Binary Pattern (LBP).

### Atributos Matriz de Coocorrência (GLCM)

Para obter os atributos provenientes da  Matriz de Coocorrência (GLCM), fizemos uso da função _[greycomatrix](https://scikit-image.org/docs/0.7.0/api/skimage.feature.texture.html)_
também da biblioteca _skimage_, essa função retorna um array que é utilizado pela função 
_greycoprops_ para obter as informações de contrast, correlation, energy e homogeneity. Visando descartar os pixels pretos, o array retornado pela primeira função foi modificado, eliminando os dados provenientes deles. Ainda, esse método pode variar de acordo com o ângulo, no entanto, para o domínio estudo essa variação não deve causar mudanças, desse modo, o GLCM foi calculado para os 4 ângulos, 0º, 45º, 90º e 135º, e os valores obtidos em cada caso foram acumulados. 


In [ ]:
# Função para extrair features da Matriz de Coocorrência (GLCM) das imagens
def get_GLCM_features(imgs: list) -> list:
  """
  ## Descrição:
    - Obtem features provenientes da GLCM a partir das imagens recebidas

  ## Entrada:
    - imgs: Lista de np.array correspondentes aos ROIs das imagens
  
  ## Saída:
    - glcm_features: Lista de dicionários contendo as features extraídas da GLCM
  """

  # Cria estrutura para armazenar as features
  glcm_features = []

  # Percorre imagens aplicando a GLCM
  for i in range(len(imgs)):
    # Cálculo das matrizes de co-ocorrência sem realizar normalização
    glcm = greycomatrix(imgs[i], distances = [1], 
                        angles = np.deg2rad([0, 45, 90, 135]), 
                        levels = 256, symmetric = True, normed = False)
    
    # Elimina pixels pretos da imagem
    glcm_br = glcm[1:, 1:, :, :]
    
    glcm_br_norm = np.true_divide(glcm_br, glcm_br.sum(axis=(0, 1)))

    # Contraste: diferença entre os tons de cinza
    contrast = greycoprops(glcm_br_norm, prop = 'contrast')

    # Correlação: dependência linear dos tons de cinza
    correlation = greycoprops(glcm_br_norm, prop = 'correlation')

    # Energia: uniformidade da textura
    energy = greycoprops(glcm_br_norm, prop = 'energy')

    # Homogeneidade: variações na textura (inversamente proporcional ao resultado)
    homogeneity = greycoprops(glcm_br_norm, prop = 'homogeneity')
    
    glcm_features.append({'contrast_sum': sum(contrast[0]),
                          'correlation_sum': sum(correlation[0]),
                          'energy_sum': sum(energy[0]),
                          'homogeneity_sum': sum(homogeneity[0])})
  
  return glcm_features

### Atributos Matriz de Comprimento de Corrida (GLRLM)

Os atributos provenientes da Matriz de Comprimento de Corrida (GLRLM) foram extraídos fazendo uso da biblioteca _[GLRLM](https://github.com/eiproject/python-glrlm)_, aqui vale ressaltar que não usamos uma função diretamente. Estudando o código da função _glrlm_ dessa biblioteca, percebemos que seria possível eliminar os pixels pretos aplicando os métodos da função de maneira independente, assim, em um passo intermediária seria possível acessar a matriz resultante e excluir os valores referentes a tais pixels, para então extrair os atributos desejados. Assim obtemos os atributos sre, lre, glu, rlu e rpc.

In [ ]:
# Função para extrair features da Matriz de Comprimento de Corrida (GLRLM) das imagens
degree  = Degree()
operator = Operator()
normalizer = Normalizer()

def get_GLRLM_features(imgs: list) -> list:
  """
  ## Descrição:
    - Obtem features provenientes da GLRLM a partir das imagens recebidas

  ## Entrada:
    - imgs: Lista de np.array correspondentes aos ROIs das imagens
  
  ## Saída:
    - glrlm_features: Lista de dicionários contendo as features extraídas da GLRLM
  """

  # Cria estrutura para armazenar as features
  glrlm_features = []
  level = 256

  # Percorre imagens aplicando a GLRLM
  for i in range(len(imgs)):
    # Execulta funções da biblioteca de maneira individual
    normalized_image = normalizer.normalizationImage(imgs[i], 0, level)
    degree_obj = degree.create_matrix(normalized_image, level)
    
    # Elimina pixels pretos da imagem
    newDegrees = []
    for mat in degree_obj.Degrees:
      newDegrees.append( mat[1:,:] )
    degree_obj.Degrees = newDegrees
    
    feature_obj = operator.create_feature(degree_obj)

    sre, lre, glu, rlu, rpc = feature_obj.Features
    glrlm_features.append({'sre': sre,
                           'lre': lre,
                           'glu': glu,
                           'rlu': rlu,
                           'rpc': rpc})
  
  return glrlm_features

### Atributos Local Binary Pattern (LBP)

As features extraídas do Local Binary Pattern (LBP) são provenientes de histogramas, de modo que após fazer uso da função _[local_binary_pattern](https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_local_binary_pattern.html)_ da biblioteca _skimage_ o mesmo processo descrito anteriormente, para a extração de features de histogramas, foi aplicado.

In [ ]:
# Função para extrair features da Local Binary Pattern (LBP) das imagens
def makeLBP(image: np.array, n_points: int, radius: int, method: str) -> np.array:
  """
  ## Descrição:
    - Aplica LBP a imagem e gera histograma a partir
    do resultado obtido

  ## Entrada:
    - image: np.array correspondentes ao ROI de uma imagem
    - n_points: int contendo o número de pontos vizinhos 
    circularmente que serão avaliados pela LBP
    - radius: int contendo o valor de raio utilizado para
    aplicação da LBP
    - method: String contendo método utilizado para
    aplicação da LBP
  
  ## Saída:
    - hist: Array contendo o histograma gerado a partir da image recebida
  """
  lbp = local_binary_pattern(image, n_points, radius, method)
  bins=30
  # Calculo histograma já eliminando os pixels pretos da imagem
  hist, x = np.histogram(lbp[lbp>0], bins) 

  return hist

def get_LBP_features(imgs: list) -> list:
  """
  ## Descrição:
    - Obtem features provenientes dos histogramas apos a aplicaçãpo
    da LBP nas imagens recebidas

  ## Entrada:
    - imgs: Lista de np.array correspondentes aos ROIs das imagens
  
  ## Saída:
    - lbp_features: Lista de dicionários contendo as features extraídas
    dos histogramas construídos após a aplicação da LBP
  """

  # Cria estrutura para armazenar as features
  lbp_features = []

  # Percorre imagens aplicando a LBP
  for i in range(len(imgs)):
    lbp_hist = makeLBP(imgs[i], 3*8, 3, 'uniform')
    describe = stats.describe(lbp_hist)._asdict()
    lbp_features.append( {'min_LBP': describe['minmax'][0],
                          'max_LBP': describe['minmax'][1],
                          'mean_LBP': describe['mean'],
                          'variance_LBP': describe['variance'],
                          'skewness_LBP': describe['skewness'],
                          'kurtosis_LBP': describe['kurtosis']} )
  
  return lbp_features

## Extração e Montagem dos Datasets

Ao final do processo de extração de atributos são gerados 3 datasets, um destinado a lesões isquêmicas(AVC), outro para desmielinizantes(EM) e um último abrigando as lesões de pacientes com Lúpus Eritematoso Sistêmico(LES). Além de trazer as features já mencionadas, os datasets também abrigam o identificador do paciente e uma feature correspondente a área da lesão, extraída durante a seleção das imagens. Ainda, para as lesões isquêmicas e desmielinizantes há um rótulo indicando esse tipo, utilizado para o treinamento do modelo. Ao final o dataset deve apresentar as informações contidas na Tabela 1.

|Coluna|Descrição|
|---|---|
|patient| Id do paciente que originou a imagem|
|min| Menor valor encontrado no histograma|
|max| Maior valor encontrado no histograma|
|mean| Média aritmética dos valores presentes no histograma|
|variance| Variância dos valores encontrados no histograma|
|skewness| Assimetria ou Obliquidade do histograma |
|kurtosis| Curtose, medida da forma que caracteriza a cauda do histograma|
|contrast_sum| Soma do contraste, diferença entre os tons de cinza, para os ângulos 0º, 45º, 90º e 135º|
|correlation_sum| Soma da correlação, dependência linear dos tons de cinza, para os ângulos 0º, 45º, 90º e 135º|
|energy_sum| Soma da energia, uniformidade da textura, para os ângulos 0º, 45º, 90º e 135º|
|homogeneity_sum| Soma da homogeneidade, medida da variação da textura, para os ângulos 0º, 45º, 90º e 135º|
|sre| Short Run Emphasis, ênfase em corrida curta 
|lre| Long Run Emphasis, ênfase em corrida longa
|glu| Grey Level Uniformity, uniformidade de tom de cinza
|rlu| Run Length Uniformity, uniformidade de tamanho de corrida
|rpc| Run Percentage, percentagem de corrida, mede predominância dos tamanhos de corrida
|min_LBP| Menor valor encontrado no histograma da LBP|
|max_LBP| Maior valor encontrado no histograma da LBP|
|mean_LBP| Média aritmética dos valores presentes no histograma da LBP|
|variance_LBP| Variância dos valores encontrado no histograma da LBP|
|skewness_LBP| Assimetria ou Obliquidade do histograma da LBP|
|kurtosis_LBP| Curtose, medida da forma que caracteriza a cauda do histograma da LBP|
|area| Número de pixels com valor maior que zero presentes nas imagem|
|target| Grupo ao qual o paciente pertence(AVC/EM)|

_Tabela 1: Atributos extraídos do conjunto de imagens._



In [ ]:
# Realiza extração de features e as retorna tabuladas em um pandas dataframe
def extract_features(roi_dict: dict, target:str) -> pd.DataFrame:
  """
  ## Descrição:
    - Realiza a extração de features, para um determinado conjunto
    imagens, fazendo uso das funções anteriores

  ## Entrada:
    - roi_dict: Dicionário de np.array, correspondentes aos ROIs das imagens,
    e suas áreas 
    - target: String com a informação do grupo de origem das imagens fornecidas
  
  ## Saída:
    - new_features: DataFrame contento todas as features extraídas para
    o conjunto de imagens fornecido
  """

  # Inicializa DataFrame
  new_features = pd.DataFrame(columns=['patient', 'min', 'max', 'mean',
                                       'variance', 'skewness', 'kurtosis',
                                       'contrast_sum', 'correlation_sum',
                                       'energy_sum', 'homogeneity_sum',
                                       'sre', 'lre', 'glu', 'rlu', 'rpc',
                                       'min_LBP', 'max_LBP', 'mean_LBP',
                                       'variance_LBP', 'skewness_LBP',
                                       'kurtosis_LBP', 'area'])
  # Percorre conjunto de pacientes
  for patient in roi_dict.keys():
    # Usa as funções anteriores para obter as features 
    glcm_features = get_GLCM_features(roi_dict[patient]['roi_imgs'])
    glrlm_features = get_GLRLM_features(roi_dict[patient]['roi_imgs'])
    hists_fetures = get_hists_features(roi_dict[patient]['roi_imgs'])
    lbp_features = get_LBP_features(roi_dict[patient]['roi_imgs'])

    # Adiciona informações ao DataFrame
    for i in range(len(roi_dict[patient]['roi_imgs'])):
      new_row = {'patient':patient, **hists_fetures[i],
                 **glcm_features[i], **glrlm_features[i], **lbp_features[i], 'area':roi_dict[patient]['areas'][i]}

      new_features = new_features.append(new_row, ignore_index=True)
    print('patient: ', patient)

  # Ordena DataFrame gerado com base nos ids dos pacientes
  new_features = new_features.sort_values(by=['patient'])

  # Adiciona coluna com o target, grupo de origem das imagens fornecidas
  if(target!='LES'):
    new_features['target'] = target
  
  return new_features

In [ ]:
# Extrai features das imagens de AVC
avc_features = extract_features(avc_roi_dict, 'AVC')

patient:  006
patient:  007
patient:  008
patient:  009
patient:  010
patient:  011
patient:  012
patient:  013
patient:  014
patient:  015
patient:  016
patient:  017


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide


patient:  018
patient:  019
patient:  020
patient:  021
patient:  022
patient:  023
patient:  024
patient:  025
patient:  026
patient:  028
patient:  029
patient:  030
patient:  031
patient:  032
patient:  033
patient:  034
patient:  035
patient:  036
patient:  037
patient:  038
patient:  039
patient:  040
patient:  041
patient:  042
patient:  043
patient:  044
patient:  045
patient:  046
patient:  047
patient:  048
patient:  049
patient:  050
patient:  051
patient:  001
patient:  002
patient:  003
patient:  004
patient:  005


In [ ]:
# Extrai features das imagens de EM
em_features = extract_features(em_roi_dict, 'EM')

patient:  060
patient:  061
patient:  062
patient:  063
patient:  064
patient:  065
patient:  066
patient:  067
patient:  068
patient:  069
patient:  070
patient:  071
patient:  072
patient:  073
patient:  074
patient:  075
patient:  076
patient:  077
patient:  078


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide


patient:  079
patient:  080
patient:  081
patient:  082
patient:  083
patient:  084
patient:  085
patient:  086
patient:  087
patient:  088
patient:  089
patient:  090
patient:  091
patient:  092
patient:  093
patient:  094
patient:  095
patient:  096
patient:  097
patient:  098
patient:  099
patient:  100
patient:  101
patient:  102
patient:  103
patient:  104
patient:  105
patient:  106
patient:  107
patient:  108
patient:  109
patient:  120


In [ ]:
# Extrai features das imagens de LES
les_features = extract_features(les_roi_dict, 'LES')

patient:  625
patient:  626
patient:  627


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide


patient:  628
patient:  629
patient:  630
patient:  631
patient:  632
patient:  633
patient:  634
patient:  635
patient:  636
patient:  637
patient:  638
patient:  639
patient:  640
patient:  641
patient:  642
patient:  643
patient:  644
patient:  645
patient:  646
patient:  647
patient:  648
patient:  649
patient:  650
patient:  651
patient:  652
patient:  653
patient:  654
patient:  655
patient:  656
patient:  657
patient:  658
patient:  659
patient:  660
patient:  661
patient:  662
patient:  663
patient:  664
patient:  665
patient:  666
patient:  667
patient:  668
patient:  669
patient:  671
patient:  672
patient:  673
patient:  674
patient:  675
patient:  676
patient:  677
patient:  678
patient:  600
patient:  601
patient:  602
patient:  603
patient:  604
patient:  605
patient:  606
patient:  607
patient:  608
patient:  609
patient:  610
patient:  611
patient:  612
patient:  613
patient:  614
patient:  615
patient:  616
patient:  617
patient:  618
patient:  619
patient:  620
patien

In [ ]:
les_features

,patient,min,max,mean,variance,skewness,kurtosis,contrast_sum,correlation_sum,energy_sum,...,glu,rlu,rpc,min_LBP,max_LBP,mean_LBP,variance_LBP,skewness_LBP,kurtosis_LBP,area
502,600,0,19,3.50,41.526316,1.542482,0.794932,38637.883965,0.657117,0.423841,...,95786.581,268.176,0.008,0,10300,344.833333,3.535281e+06,5.199438,25.034270,77
503,600,0,16,2.95,26.050000,1.426048,0.611286,405.368930,3.442599,0.464100,...,11266.245,215.454,0.012,0,5401,181.233333,9.719192e+05,5.199418,25.034135,65
507,600,0,6,0.50,2.157895,3.066279,8.365259,404.735714,1.184195,1.283656,...,12.000,40.000,0.040,0,2,0.166667,2.816092e-01,2.998808,7.389421,13
506,600,0,26,2.40,44.357895,2.824562,6.713903,222.387630,2.222683,0.529743,...,43.796,171.843,0.066,0,62,3.200000,1.278207e+02,4.919141,23.104999,53
505,600,0,12,1.90,14.200000,1.659997,1.177323,821.411667,1.972492,0.563551,...,13084.806,143.162,0.008,0,3796,127.266667,4.801310e+05,5.199427,25.034197,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,677,0,4,0.55,1.102632,2.103602,3.922477,1700.250000,0.499005,1.122164,...,10.908,44.000,0.044,0,1,0.100000,9.310345e-02,2.666667,5.111111,13
494,677,0,1,0.10,0.094737,2.666667,5.111111,NaN,NaN,NaN,...,10.000,8.000,0.016,0,0,0.000000,0.000000e+00,0.000000,-3.000000,4
497,678,0,42,6.35,132.660526,1.932754,2.792044,60541.444794,1.069504,0.318871,...,7790.279,463.812,0.022,0,1742,61.100000,1.008455e+05,5.194759,25.002608,135
495,678,0,18,5.05,38.365789,1.007466,-0.374500,7476.873001,1.283923,0.335953,...,16106.446,392.121,0.016,0,8591,289.000000,2.458638e+06,5.199396,25.033994,104


In [ ]:
# Salvar dataFrames como CSV
# avc_features.to_csv(PATH_DATA+"avc_features.csv", index=False)
# em_features.to_csv(PATH_DATA+"em_features.csv", index=False)
# les_features.to_csv(PATH_DATA+"les_features.csv", index=False)